In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("livros.csv")


In [ ]:
df.shape #linhas e colunas

In [ ]:
df.info() #tipos de cada coluna

In [71]:
traducao_colunas = {
    'Book':'livro', 
    'Author(s)':'Autor', 
    'Original language':'Idioma', 
    'First published': 'Ano',
    'Approximate sales in millions':'Vendas', 
    'Genre':'Gênero'
}

In [72]:
df.rename(columns=traducao_colunas, inplace=True)
df.columns

Index(['livro', 'Autor', 'Idioma', 'Ano', 'Vendas', 'Gênero'], dtype='object')

In [ ]:
df.isnull().sum()
#problema com a coluna gênero

In [73]:
df['Autor'].value_counts().head(10) #conta e printa os 10 maiores valores

Autor
J. K. Rowling       7
Dan Brown           3
Suzanne Collins     3
Roald Dahl          3
George Orwell       2
Ken Follett         2
E. L. James         2
Albert Camus        2
Erskine Caldwell    2
Cao Xueqin          1
Name: count, dtype: int64

In [74]:
autores_unicos = pd.DataFrame(df['Autor'].unique(), columns=['Autor'])
autores_unicos.head()

,Autor
0,Charles Dickens
1,Antoine de Saint-Exupéry
2,J. K. Rowling
3,Agatha Christie
4,Cao Xueqin


In [75]:
autores_unicos['Autor_id'] = autores_unicos.index + 1
autores_unicos.head()

,Autor,Autor_id
0,Charles Dickens,1
1,Antoine de Saint-Exupéry,2
2,J. K. Rowling,3
3,Agatha Christie,4
4,Cao Xueqin,5


In [76]:
df = df.merge(autores_unicos, on='Autor', how='left')

In [ ]:
autores = pd.DataFrame(df['Autor'].unique(), columns=['nome'])
autores.head()

In [82]:
with open('autores.sql', 'w', encoding='utf-8') as arquivo:
    for _, row in autores.iterrows():
        nome = row['nome'].replace("'", "''")
        arquivo.write(f"INSERT INTO autores (nome) VALUES ('{nome}');\n")